# Loyalty Points Analysis for Online Gaming Company (ABC)

### Importing the Liberaries

In [1]:
import pandas as pd
import numpy as np
import datetime

### Loading the Data Set 

In [2]:
# File Path
file_path = "Analytics Position Case Study.xlsx"

#Loading "Deposit Data" Sheet from the file .
deposit_df = pd.read_excel(file_path, sheet_name="Deposit Data",header=3)

#Loading "Withdrawal Data" Sheet from the file .
withdrawal_df = pd.read_excel(file_path, sheet_name="Withdrawal Data",header=3)

#Loading "User Gameplay data" Sheet from the file .
gameplay_df = pd.read_excel(file_path, sheet_name="User Gameplay data",header=3)

In [3]:
#Deposit Data Sheet
deposit_df

,User Id,Datetime,Amount
0,357,2022-01-10 00:03:00,2000
1,776,2022-01-10 00:03:00,2500
2,492,2022-01-10 00:06:00,5000
3,803,2022-01-10 00:07:00,5000
4,875,2022-01-10 00:09:00,1500
...,...,...,...
17433,654,2022-10-31 23:57:00,1200
17434,980,2022-10-31 23:58:00,200
17435,2,2022-10-31 23:58:00,40000
17436,612,2022-10-31 23:58:00,2800


In [5]:
#Withdrawal Data Sheet
withdrawal_df

,User Id,Datetime,Amount
0,190,2022-01-10 00:03:00,5872
1,159,2022-01-10 00:16:00,9540
2,164,2022-01-10 00:24:00,815
3,946,2022-01-10 00:29:00,23000
4,763,2022-01-10 00:40:00,9473
...,...,...,...
3561,559,2022-10-31 23:27:00,5000
3562,407,2022-10-31 23:51:00,3000
3563,389,2022-10-31 23:56:00,14481
3564,11,2022-10-31 23:57:00,4000


In [6]:
#gameplay Data Sheet
gameplay_df

,User ID,Games Played,Datetime
0,851,1,2022-01-10 00:00:00
1,717,1,2022-01-10 00:00:00
2,456,1,2022-01-10 00:00:00
3,424,1,2022-01-10 00:00:00
4,845,1,2022-01-10 00:00:00
...,...,...,...
355261,658,1,2022-10-31 23:59:00
355262,582,1,2022-10-31 23:59:00
355263,272,1,2022-10-31 23:59:00
355264,563,1,2022-10-31 23:59:00


### Convert Date Columns to Datetime

In [7]:
deposit_df["Datetime"] = pd.to_datetime(deposit_df["Datetime"])
withdrawal_df["Datetime"] = pd.to_datetime(withdrawal_df["Datetime"])
gameplay_df["Datetime"] = pd.to_datetime(gameplay_df["Datetime"])

### Create Slot Labels : Playerwise Loyalty Points for Specific Slots
Split each date into Slot 1 (12am–12pm) and Slot 2 (12pm–12am):

In [8]:
def assign_slot(dt):
    if dt.time() < datetime.time(12, 0, 0):
        return "S1"
    else:
        return "S2"

for df in [deposit_df, withdrawal_df, gameplay_df]:
    df["Date"] = df["Datetime"].dt.date
    df["Slot"] = df["Datetime"].apply(assign_slot)

#Alternative solution
    #df["Datetime"] = pd.to_datetime(df["Datetime"])
    #df["Date"] = df["Datetime"].dt.date
    #df["Slot"] = df["Datetime"].apply(lambda x: "S1" if x.time() < datetime.time(12, 0, 0) else "S2")

In [9]:
for dfd in [deposit_df, withdrawal_df, gameplay_df]:
    dfd["Date"] = dfd['Datetime'].dt.strftime('%Y-%m-%d')
    dfd["Time"] = dfd['Datetime'].dt.strftime('%H:%M:%S')

In [10]:
deposit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17438 entries, 0 to 17437
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   User Id   17438 non-null  int64         
 1   Datetime  17438 non-null  datetime64[ns]
 2   Amount    17438 non-null  int64         
 3   Date      17438 non-null  object        
 4   Slot      17438 non-null  object        
 5   Time      17438 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 817.5+ KB


### Filter for Target Dates

In [17]:
# ------------------------------------------------------------------
#  1. Tell the loop which date‑slot combos you need
# ------------------------------------------------------------------
# Here The Column doesn't Contain Date 2nd October . So instead of 2nd Oct we compute for 10th October . 
date_slot_pairs = [
    ("2022-10-10", "S2"),
    ("2022-10-16", "S1"),
    ("2022-10-18", "S1"),
    ("2022-10-26", "S2"),
]

# ------------------------------------------------------------------
#  2. Run the same logic for each combo and keep each result separate
# ------------------------------------------------------------------
tables = {}                    

for date, slot in date_slot_pairs:
    # --- Filter the three raw tables ------------------------------
    dep = deposit_df   .query("Date == @date and Slot == @slot")
    wit = withdrawal_df.query("Date == @date and Slot == @slot")
    gam = gameplay_df  .query("Date == @date and Slot == @slot")

    # --- Aggregate -------------------------------------------------
    dep_sum = (dep.groupby("User Id")["Amount"]
                  .agg(['sum', 'count'])
                  .rename(columns={'sum': 'Deposit_Amount',
                                   'count': 'Num_Deposits'}))

    wit_sum = (wit.groupby("User Id")["Amount"]
                  .agg(['sum', 'count'])
                  .rename(columns={'sum': 'Withdrawal_Amount',
                                   'count': 'Num_Withdrawal'}))

    gam_sum = gam.groupby("User ID").size().rename("Games_Played")

    # --- Merge and fill blanks ------------------------------------
    merged = (dep_sum
              .merge(wit_sum, left_index=True, right_index=True, how="outer")
              .merge(gam_sum, left_index=True, right_index=True, how="outer")
              .fillna(0))

    # --- Loyalty Points formula -----------------------------------
    merged["Loyalty Points"] = (
        0.01  * merged["Deposit_Amount"] +
        0.005 * merged["Withdrawal_Amount"] +
        0.001 * (merged["Num_Deposits"] - merged["Num_Withdrawal"]).clip(lower=0) +
        0.2   * merged["Games_Played"]
    )

    # --- Tidy up column names and order ---------------------------
    merged.index.name = "User ID"
    merged.reset_index(inplace=True)

    desired_order = [
        "User ID", "Games_Played",
        "Deposit_Amount", "Num_Deposits",
        "Withdrawal_Amount", "Num_Withdrawal",
        "Loyalty Points"
    ]
    merged = merged[desired_order].astype({
        "User ID": int,
        "Num_Deposits": int,
        "Num_Withdrawal": int,
        "Games_Played": int
    })

    # --- Store the slice in a dict under a clear key --------------
    tables[f"{date}_{slot}"] = merged

# ------------------------------------------------------------------
# 3. Use the individual tables however you like
# ------------------------------------------------------------------
# • Display in‑memory
#for key, df in tables.items():
   # print(f"\n=== {key} ===")
    #print(df.head())        # or display(df) in a notebook

#---------------------------------------------------------------------
#Alternative solution :
### Filter for Target Dates
#Find Playerwise Loyalty points earned by Players in the following slots:-

#    a. 2nd October Slot S1
#Since there is no date on 2nd Oct 2022 . Instead i will compute with 10 Oct 2022

#filtered_oct10_dep = deposit_df[
 #   (deposit_df["Date"] == "2022-10-10") &
  #  (deposit_df["Slot"] == "S2")]

# filtered_oct10_wit = withdrawal_df[
#    (withdrawal_df["Date"] == "2022-10-10") &
 #   (withdrawal_df["Slot"] == "S2")]

#filtered_oct10_gam = gameplay_df[
 #   (gameplay_df["Date"] == "2022-10-10") &
 #   (gameplay_df["Slot"] == "S2")]

# # Example for deposits
#deposit_summary_oct10 = filtered_oct10_dep.groupby("User Id")["Amount"].agg(["sum", "count"])
#deposit_summary_oct10.columns = ["Deposit_Amount", "Num_Deposits"]

# Example for withdrawal
#withdrawal_summary_oct10 = filtered_oct10_wit.groupby("User Id")["Amount"].agg(["sum", "count"])
#withdrawal_summary_oct10.columns = ["Withdrawal_Amount", "Num_Withdrawal"]

# Example for gameplay
#gameplay_summary_oct10 = filtered_oct10_gam.groupby("User ID").size().reset_index(name="Games_Played")
#gameplay_summary_oct10


#merged_df_oct10 = deposit_summary_oct10.merge(withdrawal_summary_oct10, on="User Id", how="outer")
#merged_df_oct10 = pd.merge(merged_df_oct10,gameplay_summary_oct10,left_on="User Id",right_on="User ID",how="outer")
#merged_df_oct10 = merged_df_oct10.fillna(0)

#merged_df_oct10["Loyalty Points"] = (
#    0.01 * merged_df_oct10["Deposit_Amount"] +
#    0.005 * merged_df_oct10["Withdrawal_Amount"] +
#    0.001 * (merged_df_oct10["Num_Deposits"] - merged_df_oct10["Num_Withdrawal"]).clip(lower=0) +
#    0.2 * merged_df_oct10["Games_Played"]
#)

#merged_df_oct10["User ID"] = merged_df_oct10["User ID"].astype(int)

#merged_df_oct10["Num_Deposits"] = merged_df_oct10["Num_Deposits"].astype(int)

#merged_df_oct10["Num_Withdrawal"] = merged_df_oct10["Num_Withdrawal"].astype(int)

#merged_df_oct10["Games_Played"] = merged_df_oct10["Games_Played"].astype(int)

#desired_order = [
#    "User ID",
#    "Games_Played",
#    "Deposit_Amount",
#    "Num_Deposits",
#    "Withdrawal_Amount",
#    "Num_Withdrawal",
#    "Loyalty Points"
#]

#merged_df_oct10 = merged_df_oct10[desired_order]

### Part A - Calculating loyalty points

1. **Find Playerwise Loyalty points earned by Players in the following slots:-**

    **a. 10th October Slot S1**
   
    **b. 16th October Slot S2**
   
    **c. 18th October Slot S1**
   
    **b. 26th October Slot S2**

**a. Playerwise Loyalty points for the Date 10th October 2022 and for Slot S2**

In [50]:
display(tables["2022-10-10_S2"])

,User ID,Games_Played,Deposit_Amount,Num_Deposits,Withdrawal_Amount,Num_Withdrawal,Loyalty Points
0,0,1,0.0,0,0.0,0,0.200
1,2,2,0.0,0,42000.0,1,210.400
2,5,5,1000.0,2,0.0,0,11.002
3,6,1,0.0,0,0.0,0,0.200
4,8,8,0.0,0,0.0,0,1.600
...,...,...,...,...,...,...,...
628,991,1,0.0,0,0.0,0,0.200
629,992,34,0.0,0,0.0,0,6.800
630,996,11,4000.0,1,0.0,0,42.201
631,997,2,0.0,0,0.0,0,0.400


Storing the Tables as per the conditions.

In [13]:
oct10_S2 = tables["2022-10-10_S2"]
oct16_S1 = tables["2022-10-16_S1"]
oct18_S1 = tables["2022-10-18_S1"]
oct26_S2 = tables["2022-10-26_S2"]

**b.Playerwise Loyalty points for the date 16th October Slot S1**


In [21]:
oct16_S1

,User ID,Games_Played,Deposit_Amount,Num_Deposits,Withdrawal_Amount,Num_Withdrawal,Loyalty Points
0,2,1,0.0,0,0.0,0,0.200
1,3,1,0.0,0,0.0,0,0.200
2,5,7,500.0,1,0.0,0,6.401
3,7,3,0.0,0,0.0,0,0.600
4,8,11,0.0,0,0.0,0,2.200
...,...,...,...,...,...,...,...
626,992,37,50000.0,1,0.0,0,507.401
627,995,1,0.0,0,0.0,0,0.200
628,996,6,0.0,0,0.0,0,1.200
629,997,3,0.0,0,0.0,0,0.600


**c. Playerwise Loyalty points for date 18th October Slot S1**

In [51]:
oct18_S1

,User ID,Games_Played,Deposit_Amount,Num_Deposits,Withdrawal_Amount,Num_Withdrawal,Loyalty Points
0,2,2,0.0,0,0.0,0,0.400
1,3,2,0.0,0,0.0,0,0.400
2,5,8,0.0,0,0.0,0,1.600
3,7,3,0.0,0,0.0,0,0.600
4,8,9,0.0,0,0.0,0,1.800
...,...,...,...,...,...,...,...
617,990,72,0.0,0,0.0,0,14.400
618,992,47,0.0,0,0.0,0,9.400
619,996,5,800.0,1,0.0,0,9.001
620,997,1,0.0,0,0.0,0,0.200


**d.Playerwise Loyalty points for date 26th October Slot S2**

In [22]:
oct26_S2

,User ID,Games_Played,Deposit_Amount,Num_Deposits,Withdrawal_Amount,Num_Withdrawal,Loyalty Points
0,2,0,90000.0,2,0.0,0,900.002
1,5,4,1000.0,1,0.0,0,10.801
2,6,1,0.0,0,0.0,0,0.200
3,7,1,0.0,0,6617.0,1,33.285
4,8,7,0.0,0,0.0,0,1.400
...,...,...,...,...,...,...,...
623,992,44,0.0,0,30000.0,1,158.800
624,995,1,0.0,0,0.0,0,0.200
625,996,4,0.0,0,0.0,0,0.800
626,997,2,0.0,0,0.0,0,0.400


**2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. 
     In case of tie, number of games played should be taken as the next criteria for ranking.**

*Filter on the Basis of month October*

In [24]:
d_oct = deposit_df[(deposit_df["Datetime"] >= pd.to_datetime("2022-10-01")) & (deposit_df["Datetime"] <= pd.to_datetime("2022-10-31"))]
w_oct = withdrawal_df[(withdrawal_df["Datetime"] >= pd.to_datetime("2022-10-01")) & (withdrawal_df["Datetime"] <= pd.to_datetime("2022-10-31"))]
g_oct = gameplay_df[(gameplay_df["Datetime"] >= pd.to_datetime("2022-10-01")) & (gameplay_df["Datetime"] <= pd.to_datetime("2022-10-31"))]


*Aggregate for month October*

In [26]:
d_agg = d_oct.groupby("User Id")["Amount"].agg(["sum", "count"]).rename(columns={"sum": "Deposit_Amount", "count": "Num_Deposit"})
w_agg = w_oct.groupby("User Id")["Amount"].agg(["sum", "count"]).rename(columns={"sum": "Withdrawal_Amount", "count": "Num_Withdrawal"})
g_agg = g_oct.groupby("User ID").size().reset_index(name="Games_Played")

*Merge all the 3 tables*

In [27]:
merged_oct = d_agg.merge(w_agg, how="outer", left_index=True, right_index=True)
merged_oct = merged_oct.merge(g_agg.set_index("User ID"), how="outer", left_index=True, right_index=True)
merged_oct = merged_oct.fillna(0)

In [31]:
merged_oct

,Deposit_Amount,Num_Deposit,Withdrawal_Amount,Num_Withdrawal,Games_Played
0,0.0,0.0,0.0,0.0,10.0
1,5000.0,1.0,0.0,0.0,5.0
2,262000.0,9.0,798466.0,12.0,63.0
3,30000.0,3.0,0.0,0.0,53.0
4,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...
995,34500.0,5.0,0.0,0.0,12.0
996,63700.0,22.0,0.0,0.0,291.0
997,0.0,0.0,0.0,0.0,65.0
998,6400.0,4.0,0.0,0.0,3.0


*Calculate Loyalty Points*

In [33]:
merged_oct["Loyalty Points"] = (
    0.01 * merged_oct["Deposit_Amount"] +
    0.005 * merged_oct["Withdrawal_Amount"] +
    0.001 * (merged_oct["Num_Deposit"] - merged_oct["Num_Withdrawal"]).clip(lower=0) +
    0.2 * merged_oct["Games_Played"]
)

*Ranking: sort by Loyalty Points, then by Games Played*

In [34]:
merged_oct["Rank"] = merged_oct.sort_values(by=["Loyalty Points", "Games_Played"], ascending=False).reset_index().index + 1

In [36]:
merged_sorted_oct = merged_oct.sort_values(by=["Loyalty Points", "Games_Played"], ascending=False)

In [37]:
merged_sorted_oct

,Deposit_Amount,Num_Deposit,Withdrawal_Amount,Num_Withdrawal,Games_Played,Loyalty Points,Rank
634,270000.0,5.0,11083419.0,47.0,21.0,58121.295,633
714,1418000.0,23.0,0.0,0.0,4.0,14180.823,713
212,1184987.0,17.0,319468.0,1.0,0.0,13447.226,211
672,1208700.0,23.0,50000.0,1.0,7.0,12338.422,671
99,769400.0,28.0,859025.0,7.0,4.0,11989.946,98
...,...,...,...,...,...,...,...
388,0.0,0.0,0.0,0.0,1.0,0.200,387
507,0.0,0.0,0.0,0.0,1.0,0.200,506
546,0.0,0.0,0.0,0.0,1.0,0.200,545
661,0.0,0.0,0.0,0.0,1.0,0.200,660


*Ranking: sort by Rank*

In [44]:
merged_sorted_oct = merged_oct.sort_values(by="Rank")

In [45]:
merged_sorted_oct

,Deposit_Amount,Num_Deposit,Withdrawal_Amount,Num_Withdrawal,Games_Played,Loyalty Points,Rank
0,0.0,0.0,0.0,0.0,10.0,2.000,1
1,5000.0,1.0,0.0,0.0,5.0,51.001,2
2,262000.0,9.0,798466.0,12.0,63.0,6624.930,3
3,30000.0,3.0,0.0,0.0,53.0,310.603,4
4,0.0,0.0,0.0,0.0,5.0,1.000,5
...,...,...,...,...,...,...,...
995,34500.0,5.0,0.0,0.0,12.0,347.405,992
996,63700.0,22.0,0.0,0.0,291.0,695.222,993
997,0.0,0.0,0.0,0.0,65.0,13.000,994
998,6400.0,4.0,0.0,0.0,3.0,64.604,995


**Question 3: Average Deposit Amount (overall)**

In [55]:
avg_deposit = deposit_df["Amount"].mean()
print("Average deposit amount:", round(avg_deposit, 2))

Average deposit amount: 5492.19


**Question 4: Average Deposit Amount per User in October**

In [42]:
avg_deposit_per_user = d_oct.groupby("User Id")["Amount"].sum().mean()
print("Average deposit per user in October:", round(avg_deposit_per_user, 2))

Average deposit per user in October: 69553.17


**Question 5: Average Number of Games Played Per User**

In [43]:
avg_games_per_user = gameplay_df.groupby("User ID").size().mean()
print("Average games per user in October:", round(avg_games_per_user, 2))

Average games per user in October: 355.27


### Part B - How much bonus should be allocated to leaderboard players?

In [63]:
# Take top 50 players
top50 = merged_sorted_oct.head(50).copy()
total_points = top50["Loyalty Points"].sum()

# Allocate 50,000 bonus proportionally
top50["Bonus"] = (top50["Loyalty Points"] / total_points) * 50000

rank50=top50[["Rank", "Loyalty Points", "Bonus"]]
rank50

,Rank,Loyalty Points,Bonus
0,1,2.000,2.317454
1,2,51.001,59.096246
2,3,6624.930,7676.486609
3,4,310.603,359.904145
4,5,1.000,1.158727
5,6,630.244,730.280867
6,7,21.603,25.031984
7,8,40.485,46.911071
8,9,63.600,73.695050
9,10,1899.503,2201.005798


### Part C – Is Loyalty Formula Fair? Suggestions

### Fairness of the Loyalty Formula:

- **Pros:**
  - Rewards all three key activities: deposit, withdrawal, and games.
  - Penalizes users who just deposit and withdraw without playing.

- **Cons:**
  - Deposit-heavy players are rewarded even if they don’t play.
  - Very low weight for withdrawals and activity frequency.

### Suggestions for Improvement:

1. **Cap Deposit Points**:
   - Prevent bonus farming by limiting deposit points to X per day or per amount.

2. **Add Win/Loss Outcomes**:
   - Consider rewarding wins or penalizing excessive losses.

3. **Activity Frequency Factor**:
   - Add points based on active days, not just volume.

4. **Bonus Multiplier**:
   - Give bonus multipliers for players who played for 7+ consecutive days.

This encourages fair usage and retention, not just big monetary moves.


Saving CSV files .

In [57]:
oct10_S2.to_csv("10th_october_slot2.csv",index=False)

In [59]:
oct16_S1.to_csv("16th_october_slot1.csv",index=False)

In [60]:
oct18_S1.to_csv("18th_october_slot1.csv",index=False)

In [61]:
oct26_S2.to_csv("26th_october_slot2.csv",index=False)

In [64]:
rank50.to_csv("rank50_players.csv",index=False)